In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
from tqdm import tqdm

2024-04-29 10:09:35.582708: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 10:09:36.055757: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
BATCH_SIZE = 512
EPOCHS = 500

initial_learning_rate = 0.001
decay_steps = 1000  # 每隔多少个steps衰减一次
decay_rate = 0.96  # 学习率衰减因子

In [3]:
AS_dataset = pd.read_csv('./hairpin_filter_dataset.csv', encoding='utf-8')

In [4]:
selected_X = AS_dataset.loc[::9,'freq':'S1'].to_numpy(dtype = np.float32)
selected_y = AS_dataset.loc[::9,'S21r':'S21i'].to_numpy(dtype = np.float32)

In [5]:
full_X = AS_dataset.loc[:,'freq':'S1'].to_numpy(dtype = np.float32)
full_y = AS_dataset.loc[:,'S21r':'S21i'].to_numpy(dtype = np.float32)

In [6]:
# 归一化
# full_X[:, 0] = (full_X[:, 0] - 0.1) / (9.0 - 0.1)
selected_X[:, 1] = (selected_X[:, 1] - 200) / (2000 - 200)
selected_X[:, 2] = (selected_X[:, 2] - 200) / (2000 - 200)
selected_X[:, 3] = (selected_X[:, 3] - 200) / (2000 - 200)
selected_X[:, 4] = (selected_X[:, 4] - 200) / (4500 - 200)
selected_X[:, 5] = (selected_X[:, 5] - 200) / (4500 - 200)
selected_X[:, 6] = (selected_X[:, 6] - 200) / (4500 - 200)
selected_X[:, 7] = (selected_X[:, 7] - 200) / (4500 - 200)
selected_X[:, 8] = (selected_X[:, 8] - 9000) / (12000 - 9000)
selected_X[:, 9] = (selected_X[:, 9] - 100) / (300 - 100)

In [7]:
X_train, X_vali, y_train, y_vali = train_test_split(selected_X, selected_y, test_size=0.05, random_state=0)

In [8]:
indices = np.random.choice(full_X.shape[0], X_vali.shape[0], replace=False)
X_of_vali = full_X[indices]
y_of_vali = full_y[indices]

In [11]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_train = dataset_train.shuffle(buffer_size=X_train.shape[0])
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2024-04-29 10:09:59.057714: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-29 10:09:59.058242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [12]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(101, 9)
        
        self.dense0a = tf.keras.layers.Dense(units=256)
        self.dense0b = tf.keras.layers.Dense(units=256)
        
        self.dense1 = tf.keras.layers.Dense(units=640, activation=tf.nn.leaky_relu)
        
        self.dense2 = tf.keras.layers.Dense(units=736, activation=tf.nn.leaky_relu)
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.dense3 = tf.keras.layers.Dense(units=672, activation=tf.nn.leaky_relu)
        
        self.dense4 = tf.keras.layers.Dense(units=576)
        self.bn4 = tf.keras.layers.BatchNormalization()
        
        self.dense5 = tf.keras.layers.Dense(units=32, activation=tf.nn.leaky_relu)
        self.bn5 = tf.keras.layers.BatchNormalization()
        
        self.dense6 = tf.keras.layers.Dense(units=2)

    def call(self, inputs):
        
        f = inputs[:, :1]
        g = inputs[:, 1:]
        x1 = self.dense0a(f)
        x2 = self.dense0b(g)
        x = x1 + x2
        
        x = self.dense1(x)

        x = self.bn2(self.dense2(x))

        x = self.dense3(x)
        
        x = self.bn4(self.dense4(x))

        x = self.bn5(self.dense5(x))

        output = self.dense6(x)
        
        return output

In [13]:
model = MLP()

In [14]:
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps,
        decay_rate,
        staircase=True  # 是否以指数方式精确衰减，默认False，若为True则每隔decay_steps学习率按decay_rate衰减
    )

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [16]:
for i in range(50):
    for X, y in tqdm(dataset_train):
        with tf.GradientTape() as tape:
            y_pred = model(X)
            tr_mse = tf.reduce_mean(tf.square(y_pred - y))
        grads = tape.gradient(tr_mse, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
        
    tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
    
    p_y_vali = model(X_vali)
    mse_vali = tf.reduce_mean(tf.square(p_y_vali - y_vali))
    r2_vali = 1 - tf.reduce_sum(tf.square(p_y_vali - y_vali)) / tf.reduce_sum(tf.square(y_vali - tf.reduce_mean(y_vali)))

    p_y_of_vali = model(X_of_vali)
    mse_of_vali = tf.reduce_mean(tf.square(p_y_of_vali - y_of_vali))
    r2_of_vali = 1 - tf.reduce_sum(tf.square(p_y_of_vali - y_of_vali)) / tf.reduce_sum(tf.square(y_of_vali - tf.reduce_mean(y_of_vali)))
    
    print(f'epoch{i}\n training loss: {tr_mse}\t training r2: {tr_r2}\n vali loss: {mse_vali}\t vali r2: {r2_vali}\n of vali loss: {mse_of_vali}\t of vali r2: {r2_of_vali}')
    
    # print(f"Epoch: {i}\t geo_seen_freq_seen(train): {tr_mse}\t geo_unseen_freq_seen: {mse_geo_unseen_freq_seen}\t geo_seen_freq_unseen: {mse_geo_seen_freq_unseen}\t geo_unseen_freq_unseen: {mse_geo_unseen_freq_unseen}")
    # log.append({'epoch': i, 'geo_seen_freq_seen': tr_mse, 'geo_unseen_freq_seen': mse_geo_unseen_freq_seen, 'geo_seen_freq_unseen': mse_geo_seen_freq_unseen, 'geo_unseen_freq_unseen': mse_geo_unseen_freq_unseen})
    # print(model.emb.variables[0][45])
    # visual(model.emb.variables[0].numpy())

  0%|                                                                                           | 0/186 [00:00<?, ?it/s]2024-04-29 10:11:17.921921: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a61265d240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-29 10:11:17.921942: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-04-29 10:11:17.921945: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-04-29 10:11:17.925312: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 10:11:18.030879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-04-29 10:11:18.133966: I ./tensorflow/compiler/jit/device_compi

100%|█████████████████████████████████████████████████████████████████████████████████| 186/186 [00:07<00:00, 26.55it/s]


epoch0
 training loss: 0.05814824253320694	 training r2: 0.00014638900756835938
 vali loss: 0.05562315136194229	 vali r2: -0.001900315284729004
 of vali loss: 1316.133056640625	 of vali r2: -21578.416015625


100%|█████████████████████████████████████████████████████████████████████████████████| 186/186 [00:04<00:00, 44.46it/s]


epoch1
 training loss: 0.07118586450815201	 training r2: -0.0032819509506225586
 vali loss: 0.05558557063341141	 vali r2: -0.001223444938659668
 of vali loss: 8886.0107421875	 of vali r2: -145694.71875


...

epoch47
 training loss: 0.005954273976385593	 training r2: 0.902385950088501
 vali loss: 0.005073193926364183	 vali r2: 0.9086201786994934
 of vali loss: 316237664.0	 of vali r2: -5185056256.0


100%|█████████████████████████████████████████████████████████████████████████████████| 186/186 [00:04<00:00, 45.65it/s]


epoch48
 training loss: 0.005300236400216818	 training r2: 0.9110198020935059
 vali loss: 0.004012223798781633	 vali r2: 0.9277306795120239
 of vali loss: 129097240.0	 of vali r2: -2116688128.0


100%|█████████████████████████████████████████████████████████████████████████████████| 186/186 [00:04<00:00, 44.50it/s]

epoch49
 training loss: 0.003595949150621891	 training r2: 0.9451079964637756
 vali loss: 0.005589987151324749	 vali r2: 0.8993115425109863
 of vali loss: 283208544.0	 of vali r2: -4643508736.0
